In [ ]:
import zarr
import matplotlib.pyplot as plt


# connect to the redis server
store = zarr.RedisStore(port=7777)

In [ ]:
def get_most_recent_key(store):
    '''helper function to get the most recent key from redis'''
    keys = list(set([s.split('/')[0] for s in store.keylist()]))
    keys.remove('.zgroup')
    keys.sort(key=int)
    print(f'current available keys: {keys}')
    return keys[-1]


# access the most recent frame of data
key = get_most_recent_key(store)
grid = zarr.open_array(store=store, mode='r', path=key)

# make a quick matplotlib plot
plt.imshow(grid, cmap='Greys')
plt.title(key)

In [ ]:
import dask.array


key = get_most_recent_key(store)
grid = zarr.open_array(store=store, mode='r', path=key)

# map the zarr array to a dask array
grid2 = dask.array.from_zarr(grid)
grid2

In [ ]:
from dask.diagnostics import ProgressBar


# compute the array using dask
with ProgressBar():
    grid2.compute()

In [ ]:
# another plot
plt.imshow(grid2, cmap='Greys')
plt.title(key)

In [ ]:
# TODO: 
#
# - checkout streamz, evaluate how "watch" the redis stream
# - checkout datashader or hvplot for